Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [ ]:
import numpy as np
import requests

API_KEY = <Insert your app key>

X = (requests.get('https://studio.edgeimpulse.com/v1/api/10419/training/28/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/10419/training/28/y', headers={'x-api-key': API_KEY})).content

Store the data in a temporary file, and load it back through Numpy.

In [ ]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [ ]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "grazing", "lying", "running", "standing", "walking" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, validation_dataset

callbacks = []


Train the model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import activations, regularizers
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import InverseTimeDecay

print(input_length)

# model architecture
model = Sequential()
# input layer
model.add(InputLayer(input_shape=(input_length, ), name='x_input'))

# hidden layer 1
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 2
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 3
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 4
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 5
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 6
model.add(Dense(64, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 7
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 8
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 9
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 10
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 11
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# hidden layer 12
model.add(Dense(32, activity_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(Dropout(0.1))

# output layer
model.add(Dense(classes, activation="softmax", name='y_pred'))

BATCH_SIZE = 64

lr_schedule = InverseTimeDecay(
  0.001,
  decay_steps=train_sample_count//BATCH_SIZE*10,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return Adam(lr_schedule)
  
# this controls the learning rate
#opt = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself

train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
model.fit(train_dataset, epochs=55, validation_data=validation_dataset, verbose=2, callbacks=callbacks)


In [ ]:
# Save the model to disk
model.save('saved_model')